In [1]:
import numpy as np
import pandas as pd
import os
import nibabel as nib
import random

In [2]:
klg_data = pd.read_csv('728_Cohort_KLG_w_Strata.csv')

In [5]:
klg_data[klg_data['Label'] == 1]

,ID,Label,Strata,Side,KLG,h5Name
3,9005132,1,145,0,4,9005132_00m_LEFT_KNEE.hdf5
9,9011918,1,202,0,3,9011918_00m_LEFT_KNEE.hdf5
11,9014797,1,282,1,1,9014797_00m_RIGHT_KNEE.hdf5
12,9014883,1,309,0,3,9014883_00m_LEFT_KNEE.hdf5
13,9015363,1,85,1,3,9015363_00m_RIGHT_KNEE.hdf5
...,...,...,...,...,...,...
717,9988891,1,127,1,3,9988891_00m_RIGHT_KNEE.hdf5
718,9991313,1,176,0,4,9991313_00m_LEFT_KNEE.hdf5
719,9992318,1,190,0,4,9992318_00m_LEFT_KNEE.hdf5
720,9992358,1,195,1,2,9992358_00m_RIGHT_KNEE.hdf5


In [ ]:
p = 'R:\OAIKNEE-Q1126\OAI_Full_Dataset\\0.C.2'
fpath_list = []
for path, subdirs, files in os.walk(p):
    while len(fpath_list) < 433:
        for name in files:
            if '_de3d1_SAG_3D_DESS_WE_LEFT.nii.gz' in name:
                pid = int(name.split('_')[0][3:])
                side = name.split('_')[-1].split('.')[0]
                left_img_path_dic[pid] = os.path.join(path, name)

In [2]:
def get_img_info(path_list):
    left_img_path_dic = {}
    right_img_path_dic = {}
    for p in path_list:
        for path, subdirs, files in os.walk(p):
            for name in files:
                if '_de3d1_SAG_3D_DESS_WE_LEFT.nii.gz' in name:
                    pid = int(name.split('_')[0][3:])
                    side = name.split('_')[-1].split('.')[0]
                    left_img_path_dic[pid] = os.path.join(path, name)
                elif '_de3d1_SAG_3D_DESS_WE_RIGHT.nii.gz' in name:
                    pid = int(name.split('_')[0][3:])
                    side = name.split('_')[-1].split('.')[0]
                    right_img_path_dic[pid] = os.path.join(path, name)
    return left_img_path_dic, right_img_path_dic

def match(left_pid_set, right_pid_set, left_dic, right_dic):
    target_fp = []
    target_id = set()
    for id in left_pid_set:
        if id in left_dic:
            target_id.add(id)
            target_fp.append(left_dic[id])
    for id in right_pid_set:
        if id in right_dic:
            target_id.add(id)
            target_fp.append(right_dic[id])
    return target_fp, target_id

klg_data = pd.read_csv('728_Cohort_KLG_w_Strata.csv')
left_dic, right_dic = get_img_info(['R:\OAIKNEE-Q1126\OAI_Full_Dataset\\0.C.2', 'R:\OAIKNEE-Q1126\OAI_Full_Dataset\\0.E.1'])
with open('Outcomes99.txt') as f:
    lines = f.readlines()
left_pids = []
right_pids = []
for line in lines:
    splited = line.split('|')
    if 'Total' in splited[6] or 'Partial' in splited[6]:
        right_pids.append(int(splited[0]))
    if 'Total' in splited[20] or 'Partial' in splited[20]:
        left_pids.append(int(splited[0]))

In [3]:
def get_no_tkr_imgs(left_pid_set, right_pid_set, left_dic, right_dic):
    target_fp = []
    for id in left_dic:
        if id not in left_pid_set:
            target_fp.append(left_dic[id])
    for id in right_dic:
        if id not in right_pid_set:
            target_fp.append(right_dic[id])
    return target_fp

In [4]:
left_pid_set = set(left_pids)
right_pid_set = set(right_pids)
for index, row in klg_data.iterrows():
    if row[1] == 1 and row[3] == 0:
        left_pid_set.add(row[0])
    if row[1] == 1 and row[3] == 1:
        right_pid_set.add(row[0])
        
non_tkf_fp = get_no_tkr_imgs(left_pid_set, right_pid_set, left_dic, right_dic)

In [11]:
len(non_tkf_fp)

6763

In [5]:
non_tkf_fp_random = random.choices(non_tkf_fp, k=433)

In [6]:
import shutil
for fp in non_tkf_fp_random:
    shutil.copy(fp, 'R:\OAIKNEE-Q1126\OAI_UQ_processed\AKOA_TKR_Data\\NO_TKR')